# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm

In [2]:
data = pd.read_csv('../previsao_de_renda.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.dropna(inplace=True)
data.head()

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               12427 non-null  object 
 1   id_cliente             12427 non-null  int64  
 2   sexo                   12427 non-null  object 
 3   posse_de_veiculo       12427 non-null  bool   
 4   posse_de_imovel        12427 non-null  bool   
 5   qtd_filhos             12427 non-null  int64  
 6   tipo_renda             12427 non-null  object 
 7   educacao               12427 non-null  object 
 8   estado_civil           12427 non-null  object 
 9   tipo_residencia        12427 non-null  object 
 10  idade                  12427 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  12427 non-null  float64
 13  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
def casela_de_ref(col: pd.Series) -> str:
    index = ''
    freq_lst = col.value_counts() / len(col)
    for i, value in enumerate(freq_lst):
        if value == freq_lst.max():
            index = freq_lst.index[i]
    return index

In [5]:
formula = 'np.log(renda) ~ '

for col in data.loc[:, data.dtypes == 'object']:
    formula += "C({}, Treatment(reference='{}')) + ".format(col, casela_de_ref(data[col]))

formula = formula[:-3]
formula

"np.log(renda) ~ C(data_ref, Treatment(reference='2015-09-01')) + C(sexo, Treatment(reference='F')) + C(tipo_renda, Treatment(reference='Assalariado')) + C(educacao, Treatment(reference='Secundário')) + C(estado_civil, Treatment(reference='Casado')) + C(tipo_residencia, Treatment(reference='Casa'))"

In [6]:
y, X = patsy.dmatrices(formula, data)
reg1 = sm.OLS(y, X).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     63.47
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        20:18:55   Log-Likelihood:                -15373.
No. Observations:               12427   AIC:                         3.081e+04
Df Residuals:                   12394   BIC:                         3.106e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 7.9373      0.031    253.043      0.000       7.876       7.999
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-01-01]              0.0150      0.041      0.368      0.713      -0.065       0.095
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-02-01]              0.0031      0.041      0.075      0.940      -0.077       0.083
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-03-01]              0.0472      0.041      1.153      0.249      -0.033       0.128
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-04-01]              0.0796      0.041      1.949      0.051      -0.000       0.160
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-05-01]             -0.0216      0.041     -0.530      0.596      -0.101       0.058
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-06-01]              0.0845      0.041      2.068      0.039       0.004       0.165
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-07-01]              0.0320      0.041      0.785      0.433      -0.048       0.112
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-08-01]              0.0169      0.041      0.412      0.680      -0.063       0.097
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-10-01]              0.0410      0.041      1.007      0.314      -0.039       0.121
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-11-01]              0.0320      0.041      0.786      0.432      -0.048       0.112
C(data_ref, Treatment(reference='2015-09-01'))[T.2015-12-01]              0.0826      0.041      2.021      0.043       0.002       0.163
C(data_ref, Treatment(reference='2015-09-01'))[T.2016-01-01]             -0.0094      0.041     -0.230      0.818      -0.089       0.070
C(data_ref, Treatment(reference='2015-09-01'))[T.2016-02-01]              0.0321      0.041      0.790      0.430      -0.048       0.112
C(data_ref, Treatment(reference='2015-09-01'))[T.2016-03-01]             -0.0361      0.041     -0.878      0.380      -0.117       0.045
C(sexo, Treatment(reference='F'))[T.M]                                    0.6842      0.016     43.013      0.000       0.653       0.715
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]             0.4057      0.279      1.455      0.146      -0.141       0.952
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.0708      0.017      4.117      0.000       0.037       0.104
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.3881      0.279     -1.390      0.164      -0.935       0.159
C(tipo_renda, Treatment(reference='Assalariado'

A única variável sem sentido prático para usar na criação do modelo é a variável **data_ref**. Portanto, vamos criar um novo modelo que exclua essa variável. 

In [8]:
new_formula = "np.log(renda) ~ C(sexo, Treatment(reference='F')) + C(tipo_renda, Treatment(reference='Assalariado')) + C(educacao, Treatment(reference='Secundário')) + C(estado_civil, Treatment(reference='Casado')) + C(tipo_residencia, Treatment(reference='Casa'))"

y, X = patsy.dmatrices(new_formula, data)
reg2 = sm.OLS(y, X).fit()
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.138
Method:                 Least Squares   F-statistic:                     111.5
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        20:21:52   Log-Likelihood:                -15384.
No. Observations:               12427   AIC:                         3.081e+04
Df Residuals:                   12408   BIC:                         3.095e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 7.9650      0.014    565.802      0.000       7.937       7.993
C(sexo, Treatment(reference='F'))[T.M]                                    0.6834      0.016     42.962      0.000       0.652       0.715
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]             0.4080      0.279      1.463      0.143      -0.139       0.955
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.0695      0.017      4.042      0.000       0.036       0.103
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.3896      0.279     -1.396      0.163      -0.937       0.157
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]     0.2161      0.026      8.460      0.000       0.166       0.266
C(educacao, Treatment(reference='Secundário'))[T.Primário]               -0.0708      0.083     -0.850      0.395      -0.234       0.092
C(educacao, Treatment(reference='Secundário'))[T.Pós graduação]          -0.0764      0.164     -0.465      0.642      -0.398       0.246
C(educacao, Treatment(reference='Secundário'))[T.Superior completo]       0.0808      0.016      5.073      0.000       0.050       0.112
C(educacao, Treatment(reference='Secundário'))[T.Superior incompleto]    -0.1266      0.037     -3.431      0.001      -0.199      -0.054
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                0.0671      0.032      2.087      0.037       0.004       0.130
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]               -0.0335      0.023     -1.441      0.150      -0.079       0.012
C(estado_civil, Treatment(reference='Casado'))[T.União]                  -0.0737      0.029     -2.545      0.011      -0.131      -0.017
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                   0.1286      0.048      2.694      0.007       0.035       0.222
C(tipo_residencia, Treatment(reference='Casa'))[T.Aluguel]               -0.1178      0.063     -1.884      0.060      -0.240       0.005
C(tipo_residencia, Treatment(reference='Casa'))[T.Com os pais]           -0.1583      0.033     -4.754      0.000      -0.224      -0.093
C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]           -0.2420      0.105     -2.300      0.021      -0.448      -0.036
C(tipo_residencia, Treatment(reference='Casa'))[T.Estúdio]                0.2164      0.097      2.230      0.026       0.026       0.407
C(tipo_residencia, Treatment(reference='Casa'))[T.Governamental]         -0.0203      0.045     -0.454      0.650      -0.108       0.067
===============================================

A remoção da variável não alterou muito o desempenho do modelo, não houve uma melhora no **AIC** e houve uma pequena queda já esperada no **R²** ao remover uma variável.

A variável menos significante do modelo atual é **educacao**, vamos criar um modelo que remova essa variável.

In [9]:
y, X = patsy.dmatrices("np.log(renda) ~ C(sexo, Treatment(reference='F')) + C(tipo_renda, Treatment(reference='Assalariado')) + C(estado_civil, Treatment(reference='Casado')) + C(tipo_residencia, Treatment(reference='Casa'))"
, data)
reg3 = sm.OLS(y, X).fit()
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     139.6
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        20:24:58   Log-Likelihood:                -15408.
No. Observations:               12427   AIC:                         3.085e+04
Df Residuals:                   12412   BIC:                         3.096e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 7.9874      0.013    624.913      0.000       7.962       8.012
C(sexo, Treatment(reference='F'))[T.M]                                    0.6797      0.016     42.693      0.000       0.648       0.711
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]             0.4664      0.279      1.671      0.095      -0.081       1.014
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.0764      0.017      4.460      0.000       0.043       0.110
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.3377      0.279     -1.208      0.227      -0.885       0.210
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]     0.2316      0.025      9.098      0.000       0.182       0.282
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                0.0716      0.032      2.223      0.026       0.008       0.135
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]               -0.0346      0.023     -1.491      0.136      -0.080       0.011
C(estado_civil, Treatment(reference='Casado'))[T.União]                  -0.0776      0.029     -2.674      0.007      -0.134      -0.021
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                   0.1267      0.048      2.652      0.008       0.033       0.220
C(tipo_residencia, Treatment(reference='Casa'))[T.Aluguel]               -0.1276      0.063     -2.038      0.042      -0.250      -0.005
C(tipo_residencia, Treatment(reference='Casa'))[T.Com os pais]           -0.1621      0.033     -4.868      0.000      -0.227      -0.097
C(tipo_residencia, Treatment(reference='Casa'))[T.Comunitário]           -0.2256      0.105     -2.150      0.032      -0.431      -0.020
C(tipo_residencia, Treatment(reference='Casa'))[T.Estúdio]                0.2295      0.097      2.361      0.018       0.039       0.420
C(tipo_residencia, Treatment(reference='Casa'))[T.Governamental]         -0.0267      0.045     -0.595      0.552      -0.115       0.061
==============================================================================
Omnibus:                      135.220   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              148.296
Skew:                           0.221   Prob(JB):                     6.28e-33
Kurtosis:                       3.301   Cond. No.                         42.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Agora a variável de menor significância é **tipo_residencia**, vamos criar um outro modelo sem a variável.

In [10]:
y, X = patsy.dmatrices("np.log(renda) ~ C(sexo, Treatment(reference='F')) + C(tipo_renda, Treatment(reference='Assalariado')) + C(estado_civil, Treatment(reference='Casado'))"
, data)
reg4 = sm.OLS(y, X).fit()
reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     212.4
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        20:26:30   Log-Likelihood:                -15427.
No. Observations:               12427   AIC:                         3.087e+04
Df Residuals:                   12417   BIC:                         3.095e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 7.9769      0.013    634.218      0.000       7.952       8.002
C(sexo, Treatment(reference='F'))[T.M]                                    0.6765      0.016     42.622      0.000       0.645       0.708
C(tipo_renda, Treatment(reference='Assalariado'))[T.Bolsista]             0.4769      0.279      1.706      0.088      -0.071       1.025
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.0808      0.017      4.714      0.000       0.047       0.114
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.3199      0.280     -1.143      0.253      -0.868       0.229
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]     0.2350      0.025      9.231      0.000       0.185       0.285
C(estado_civil, Treatment(reference='Casado'))[T.Separado]                0.0696      0.032      2.161      0.031       0.006       0.133
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]               -0.0423      0.023     -1.822      0.068      -0.088       0.003
C(estado_civil, Treatment(reference='Casado'))[T.União]                  -0.0803      0.029     -2.767      0.006      -0.137      -0.023
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]                   0.1292      0.048      2.703      0.007       0.036       0.223
==============================================================================
Omnibus:                      141.070   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              155.611
Skew:                           0.225   Prob(JB):                     1.62e-34
Kurtosis:                       3.314   Cond. No.                         42.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Agora vamos remover a variável **tipo_renda**.

In [11]:
y, X = patsy.dmatrices("np.log(renda) ~ C(sexo, Treatment(reference='F')) + C(estado_civil, Treatment(reference='Casado'))"
, data)
reg5 = sm.OLS(y, X).fit()
reg5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     360.1
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        20:27:37   Log-Likelihood:                -15475.
No. Observations:               12427   AIC:                         3.096e+04
Df Residuals:                   12421   BIC:                         3.101e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------
Intercept                                                      8.0275      0.011    730.602      0.000       8.006       8.049
C(sexo, Treatment(reference='F'))[T.M]                         0.6657      0.016     41.894      0.000       0.635       0.697
C(estado_civil, Treatment(reference='Casado'))[T.Separado]     0.0636      0.032      1.967      0.049       0.000       0.127
C(estado_civil, Treatment(reference='Casado'))[T.Solteiro]    -0.0386      0.023     -1.661      0.097      -0.084       0.007
C(estado_civil, Treatment(reference='Casado'))[T.União]       -0.0795      0.029     -2.731      0.006      -0.137      -0.022
C(estado_civil, Treatment(reference='Casado'))[T.Viúvo]        0.1320      0.048      2.753      0.006       0.038       0.226
==============================================================================
Omnibus:                      145.885   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              159.388
Skew:                           0.233   Prob(JB):                     2.45e-35
Kurtosis:                       3.300   Cond. No.                         6.99
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""